# Reinforcement Learning in ConnectX

The following notebook follows the third step of the tutorial. It should provide an introduction to submitting reinforcement learning solutions to Kaggle. This notebook does not act as a detailed guide. For more detail, see my other notebook on ConnectX or the Intro to Game AI and Reinforcement Learning course.

# Implement the game environment
We want to do two things in this notebook:
1. Test and edit our agents
2. Submit our solution

To test and edit our agents, we have to set up the test environment as shown in the Reinforcement Learning course.

In [ ]:
# Here we import standard libraries and our environment
# You must first add the data for the task in the settings column
import random
import numpy as np
from kaggle_environments import make, evaluate

# Create the game environment
env = make("connectx", debug=True)

# List the available agents
print(list(env.agents))

Let's check this works by running two random agents against each other and rendering to watch the game.

In [ ]:
env.run(["random", "random"])

# To render using iPython, we have to use a notebook as the Kaggle editor can't show HTML objects
env.render(mode="ipython")

# Helping Functions
Before defining our agent, we first define some functions that determine how a given move might affect play.

In [ ]:
# Gets board at next step if agent drops piece in selected column
def drop_piece(grid, col, piece, config):
    next_grid = grid.copy()
    for row in range(config.rows-1, -1, -1):
        if next_grid[row][col] == 0:
            break
    next_grid[row][col] = piece
    return next_grid

# Helper function for get_heuristic: counts number of windows satisfying specified heuristic conditions
def count_windows(grid, num_discs, piece, config):
    num_windows = 0
    # horizontal
    for row in range(config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[row, col:col+config.inarow])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # vertical
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns):
            window = list(grid[row:row+config.inarow, col])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # positive diagonal
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # negative diagonal
    for row in range(config.inarow-1, config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    return num_windows

# Additional Functions

In [ ]:
# Helper function for minimax: checks if agent or opponent has four in a row in the window
def is_terminal_window(window, config):
    return window.count(1) == config.inarow or window.count(2) == config.inarow

# Helper function for minimax: checks if game has ended
def is_terminal_node(grid, config):
    # Check for draw 
    if list(grid[0, :]).count(0) == 0:
        return True
    # Check for win: horizontal, vertical, or diagonal
    # horizontal 
    for row in range(config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[row, col:col+config.inarow])
            if is_terminal_window(window, config):
                return True
    # vertical
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns):
            window = list(grid[row:row+config.inarow, col])
            if is_terminal_window(window, config):
                return True
    # positive diagonal
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
            if is_terminal_window(window, config):
                return True
    # negative diagonal
    for row in range(config.inarow-1, config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
            if is_terminal_window(window, config):
                return True
    return False

Finally we need to define a heuristic to rank our move

In [ ]:
# Helper function for get_heuristic: checks if window satisfies heuristic conditions
def check_window(window, num_discs, piece, config):
    return (window.count(piece) == num_discs and window.count(0) == config.inarow-num_discs)

def get_heuristic(grid, mark, config):
    num_threes = count_windows(grid, 3, mark, config)
    num_fours = count_windows(grid, 4, mark, config)
    num_threes_opp = count_windows(grid, 3, mark%2+1, config)
    num_fours_opp = count_windows(grid, 4, mark%2+1, config)
    score = num_threes - 1e2*num_threes_opp - 1e4*num_fours_opp + 1e6*num_fours
    return score

# Minimax Agent

In [ ]:
def minimax(node, depth, maximizingPlayer, mark, config):
    is_terminal = is_terminal_node(node, config)
    valid_moves = [c for c in range(config.columns) if node[0][c] == 0]
    if depth == 0 or is_terminal:
        return get_heuristic(node, mark, config)
    if maximizingPlayer:
        value = -np.Inf
        for col in valid_moves:
            child = drop_piece(node, col, mark, config)
            value = max(value, minimax(child, depth-1, False, mark, config))
        return value
    else:
        value = np.Inf
        for col in valid_moves:
            child = drop_piece(node, col, mark%2+1, config)
            value = min(value, minimax(child, depth-1, True, mark, config))
        return value

# Uses minimax to calculate value of dropping piece in selected column
def score_move(grid, col, mark, config, nsteps):
    next_grid = drop_piece(grid, col, mark, config)
    score = minimax(next_grid, nsteps-1, False, mark, config)
    return score

In [ ]:
def agent_minimax(obs, config):
    # How deep to make the game tree: higher values take longer to run!
    N_STEPS = 3
    # Get list of valid moves
    valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]
    # Convert the board to a 2D grid
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    # Use the heuristic to assign a score to each possible board in the next step
    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config, N_STEPS) for col in valid_moves]))
    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    # Select at random from the maximizing columns
    return random.choice(max_cols)

# Test the Agent

In [ ]:
# Run once to observe the procedure is implemented correctly
env.run([agent_minimax, "random"])
env.render(mode="ipython")

# Determine the winning percentages with
# get_win_percentages(agent1=agent_blocker, agent2=agent_random)

# Determine the Winning Perccentages

In [ ]:
def get_win_percentages(agent1, agent2, n_rounds=100):
    # Use default Connect Four setup
    config = {'rows': 6, 'columns': 7, 'inarow': 4}
    # Agent 1 goes first (roughly) half the time          
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds//2)
    # Agent 2 goes first (roughly) half the time      
    outcomes += [[b,a] for [a,b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds-n_rounds//2)]
    print("Agent 1 Win Percentage:", np.round(outcomes.count([1,-1])/len(outcomes), 2))
    print("Agent 2 Win Percentage:", np.round(outcomes.count([-1,1])/len(outcomes), 2))
    print("Number of Invalid Plays by Agent 1:", outcomes.count([None, 0]))
    print("Number of Invalid Plays by Agent 2:", outcomes.count([0, None]))

# Generate the submission.py file
You may notice the following script is different to that in the course. Hopefully this is more intuitive.

First, we open our file "submission.py". Then, we search through our notebook and one-by-one add the functions needed for our agent. Finally we close our file.

Suppose we wanted to add our leftmost agent, we would use the following script.

In [ ]:
import inspect
import os

f = open("submission.py", "w")
f.write("import random \n")
f.write("import numpy as np \n")
f.write(inspect.getsource(drop_piece))
f.write(inspect.getsource(count_windows))
f.write(inspect.getsource(is_terminal_window))
f.write(inspect.getsource(is_terminal_node))
f.write(inspect.getsource(check_window))
f.write(inspect.getsource(get_heuristic))
f.write(inspect.getsource(minimax))
f.write(inspect.getsource(score_move))
f.write(inspect.getsource(agent_minimax))
f.close()

print("agent_minimax", "written to", "submission.py")

To check this has worked, look in the tab to the right. Under data, find the "output" folder. Find your "submission.py" file and download it to observe the contents. We should have a self-contained script with all necessary functions for our agent. Now, we are ready to submit this solution!

# Submitting to the Competition
I want to refer back to the course here as it offers a very good explanation of how to make our final submission.

1. Begin by clicking on the blue Save Version button in the top right corner of the window. This will generate a pop-up window.
2. Ensure that the Save and Run All option is selected, and then click on the blue Save button.
3. This generates a window in the bottom left corner of the notebook. After it has finished running, click on the number to the right of the Save Version button. This pulls up a list of versions on the right of the screen. Click on the ellipsis (...) to the right of the most recent version, and select Open in Viewer. This brings you into view mode of the same page. You will need to scroll down to get back to these instructions.
4. Click on the Output tab on the right of the screen. Then, click on the blue Submit button to submit your results to the leaderboard.

And you've submitted!!! :)

I hope you all found success with this notebook. If you're still having difficulties submitting, feel free to comment on this notebook and I'll reply as soon as I can.